In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('Known StoryBuilt properties in Texas - Sheet1.csv')

In [7]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address'].apply(geocode)

In [8]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [11]:
df.columns

Index(['Address', 'Name of Property', 'Description', 'Link', 'Status',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [13]:
df["short_description"] = df["Description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://bruno.storybuilt.com/" target...
1     <a href="https://charley.storybuilt.com/" targ...
2     <a href="https://clementine.storybuilt.com/" t...
3     <a href="https://www.storybuilt.com/cities/aus...
4     <a href="https://frank.storybuilt.com/" target...
5     <a href="https://www.liveatgeorgeatx.com/" tar...
6     <a href="https://goose.storybuilt.com/" target...
7     <a href="https://elliemay.storybuilt.com/" tar...
8     <a href="https://design.storybuilt.com/project...
9     <a href="https://www.storybuilt.com/wp-content...
10    <a href="https://www.storybuilt.com/2022/03/lu...
11    <a href="https://www.storybuilt.com/wp-content...
12    <a href="https://ruby.storybuilt.com/" target=...
13    <a href="https://www.thorntonflats.com/contact...
14    <a href="https://www.storybuilt.com/cities/aus...
15    <a href="https://oakcliff.advocatemag.com/2021...
16    <a href="https://www.storybuilt.com/cities/dal...
Name: description_link, dtype: object

In [14]:
df['description_link']

0     <a href="https://bruno.storybuilt.com/" target...
1     <a href="https://charley.storybuilt.com/" targ...
2     <a href="https://clementine.storybuilt.com/" t...
3     <a href="https://www.storybuilt.com/cities/aus...
4     <a href="https://frank.storybuilt.com/" target...
5     <a href="https://www.liveatgeorgeatx.com/" tar...
6     <a href="https://goose.storybuilt.com/" target...
7     <a href="https://elliemay.storybuilt.com/" tar...
8     <a href="https://design.storybuilt.com/project...
9     <a href="https://www.storybuilt.com/wp-content...
10    <a href="https://www.storybuilt.com/2022/03/lu...
11    <a href="https://www.storybuilt.com/wp-content...
12    <a href="https://ruby.storybuilt.com/" target=...
13    <a href="https://www.thorntonflats.com/contact...
14    <a href="https://www.storybuilt.com/cities/aus...
15    <a href="https://oakcliff.advocatemag.com/2021...
16    <a href="https://www.storybuilt.com/cities/dal...
Name: description_link, dtype: object

In [17]:
df.columns

Index(['Address', 'Name of Property', 'Description', 'Link', 'Status',
       'geocoded', 'lat', 'lon', 'short_description', 'remaining_desc',
       'description_link'],
      dtype='object')

In [32]:
def popup_html(row):
    i = row
    Address = df['Address'].iloc[i]
    Name_of_Property = df['Name of Property'].iloc[i]
    Status = df['Status'].iloc[i]
    Description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>''' + \
    '''<strong>Address: </strong>{}<br>
    <strong>Property Name: </strong>{}<br>
    <strong>Status: </strong>{}<br>
    <strong>Description: </strong>{}'''.format(Address, Name_of_Property, Status, Description) + \
    '''
    </html>
    '''
    return html


In [39]:
import folium
import branca
from folium.plugins import MarkerCluster

# Sample dataframe
# df = ...

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=4)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Known StoryBuilt Properties')

# Create a MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for i in range(0, len(df)):
    html = popup_html(i)  # Assuming popup_html function is defined elsewhere
    iframe = branca.element.IFrame(html=html, width=256)
    popup = folium.Popup(folium.Html(html, script=True), max_width=256)
    
    # Add markers to the marker_cluster instead of directly to the map
    folium.Marker(
        [df['lat'].iloc[i], df['lon'].iloc[i]],
        popup=popup
    ).add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].values.tolist())
# m.add_to(f)  # If you need to add the map to a folium Figure
m

In [40]:
m.save('index.html')

## Map URL Snagger

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Known_StoryBuilt_Properties
